## 1) Loading and checking the dataset

In [90]:
import pandas as pd 

In [140]:
places= pd.read_csv('/Users/workplace/Documents/GitHub/Neighborhood-Insight-Qurtubah/data/places.csv')

In [141]:
places.shape

(2145, 8)

In [142]:
places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145 entries, 0 to 2144
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         2145 non-null   object 
 1   category      2113 non-null   object 
 2   dist          1871 non-null   object 
 3   lat           2086 non-null   float64
 4   lng           2086 non-null   float64
 5   rating        1929 non-null   float64
 6   reviewsCount  2085 non-null   float64
 7   price         288 non-null    object 
dtypes: float64(4), object(4)
memory usage: 134.2+ KB


In [143]:
places.head()

,title,category,dist,lat,lng,rating,reviewsCount,price
0,أسواق ثمرات الجنتين,Supermarket,"RQ85+RX Al Munsiyah, Riyadh Saudi Arabia",24.817080,46.759923,3.6,197.0,NaN
1,Meed Store,Convenience store,NaN,24.788248,46.738404,3.9,81.0,NaN
2,Alsultan mart,Supermarket,"RQ85+J7 Al Munsiyah, Riyadh Saudi Arabia",24.816546,46.758185,4.0,253.0,NaN
3,Nespresso,Coffee vending machine,NaN,24.757320,46.630134,4.2,351.0,NaN
4,Wadi square,Shopping mall,"QPQ2+RG Al Wadi, Riyadh Saudi Arabia",24.789562,46.701330,4.5,4.0,NaN


## 2) Data Cleaning 

check for duplicates 

In [144]:
places.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2140     True
2141     True
2142     True
2143     True
2144     True
Length: 2145, dtype: bool

In [145]:
places.drop_duplicates(inplace=True)

In [146]:
places.shape

(2137, 8)

Check for nulls 

In [147]:
places.isna().sum()

title              0
category          32
dist             274
lat               59
lng               59
rating           208
reviewsCount      52
price           1849
dtype: int64

- for rating and reviewscount I'll leave nulls, becuase these places hasn't been reviewd yet.
- for long and lat I'll replace the missing values with the lat and lng of Qurtubah.
- for the nulls in dist I'll calculate the distance in kms using lat and lng.  
- I'll leave the nulls in price column as they are.


In [148]:
neighborhood_latitude=24.8166
neighborhood_longitude=46.7320

In [149]:
#method to calculate distance.
# 
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [150]:
places['distance'] = places.apply(lambda row: distance((float(neighborhood_latitude),float(neighborhood_longitude)),(row['lat'],row['lng'])), axis=1)

In [151]:
places.head()

,title,category,dist,lat,lng,rating,reviewsCount,price,distance
0,أسواق ثمرات الجنتين,Supermarket,"RQ85+RX Al Munsiyah, Riyadh Saudi Arabia",24.817080,46.759923,3.6,197.0,NaN,2.818646
1,Meed Store,Convenience store,NaN,24.788248,46.738404,3.9,81.0,NaN,3.218145
2,Alsultan mart,Supermarket,"RQ85+J7 Al Munsiyah, Riyadh Saudi Arabia",24.816546,46.758185,4.0,253.0,NaN,2.642804
3,Nespresso,Coffee vending machine,NaN,24.757320,46.630134,4.2,351.0,NaN,12.214699
4,Wadi square,Shopping mall,"QPQ2+RG Al Wadi, Riyadh Saudi Arabia",24.789562,46.701330,4.5,4.0,NaN,4.315402


In [173]:
places[(places['dist'].isna()) & (places['distance']<=2.2)]

,title,category,dist,lat,lng,rating,reviewsCount,price,distance
8,Abdullah Alothaim Markets,Supermarket,NaN,24.821437,46.742188,3.8,215.0,NaN,1.160378
15,مانويل ماركت,Supermarket,NaN,24.829196,46.726967,5.0,2.0,NaN,1.489884
26,Panda Super market Qurtubah,Supermarket,NaN,24.813905,46.724659,1.0,1.0,NaN,0.799264
39,BAJA,Supermarket,NaN,24.811075,46.734269,4.7,7.0,NaN,0.655678
77,looz & Jooz,Nut store,NaN,24.814830,46.726020,4.1,42.0,NaN,0.634871
...,...,...,...,...,...,...,...,...,...
1853,Mini Mart,Grocery store,NaN,24.808720,46.732702,NaN,0.0,NaN,0.879101
1854,طاحون حسين,Grocery store,NaN,24.818429,46.752263,3.9,10.0,NaN,2.055147
1857,comilla super market,Grocery store,NaN,24.806166,46.720211,5.0,1.0,NaN,1.661825
1859,Thimar Alqassim Super Market,Grocery store,NaN,24.814820,46.753083,3.7,21.0,NaN,2.137049


In [171]:
#places['dist'].sample(10)

257           QMGF+WR An Nafal, Riyadh Saudi Arabia
1196       RQ83+XM Al Munsiyah, Riyadh Saudi Arabia
378           QQH3+VW Al Hamra, Riyadh Saudi Arabia
1301    MPR7+4C As Sulimaniyah, Riyadh Saudi Arabia
921           RQ72+46 Qurtubah, Riyadh Saudi Arabia
1550          MMGR+QW Al Olaya, Riyadh Saudi Arabia
2077                  طريق الشيخ حسن بن حسين بن علي
1780          RP46+C5 Al Falah, Riyadh Saudi Arabia
1210         QPWV+4P Ghirnatah, Riyadh Saudi Arabia
2108                               Qurtubah, Riyadh
Name: dist, dtype: object

In [172]:
#places[(places['dist'].isna()) & (places['distance']<=2.2)]

,title,category,dist,lat,lng,rating,reviewsCount,price,distance
8,Abdullah Alothaim Markets,Supermarket,NaN,24.821437,46.742188,3.8,215.0,NaN,1.160378
15,مانويل ماركت,Supermarket,NaN,24.829196,46.726967,5.0,2.0,NaN,1.489884
26,Panda Super market Qurtubah,Supermarket,NaN,24.813905,46.724659,1.0,1.0,NaN,0.799264
39,BAJA,Supermarket,NaN,24.811075,46.734269,4.7,7.0,NaN,0.655678
77,looz & Jooz,Nut store,NaN,24.814830,46.726020,4.1,42.0,NaN,0.634871
...,...,...,...,...,...,...,...,...,...
1853,Mini Mart,Grocery store,NaN,24.808720,46.732702,NaN,0.0,NaN,0.879101
1854,طاحون حسين,Grocery store,NaN,24.818429,46.752263,3.9,10.0,NaN,2.055147
1857,comilla super market,Grocery store,NaN,24.806166,46.720211,5.0,1.0,NaN,1.661825
1859,Thimar Alqassim Super Market,Grocery store,NaN,24.814820,46.753083,3.7,21.0,NaN,2.137049


## Project Quiestions 

### A) How diverse are the destinations in Qurtubah?

### B)  What are the best destinations?

### C) How family friendly is the neighborhood ?

### D) How is rent affected in comparsion to other neighborhoods?

### E) How walkable is Qurtubah?